In [1]:
import argparse
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import yaml

import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from torchvision import transforms, utils

from datasets import *
from nets import *
from functions import *
from trainer import *

In [26]:
import pandas as pd

df = pd.read_csv('data/fiw_labels.txt', sep=' ', header=None, names=['index', 'filename', 'filename2', 'kt', 'gt'])
df = df.drop(['index', 'filename2', 'kt', 'gt'], axis=1).drop_duplicates()
df.to_csv('fiw_files.csv')

,filename
0,Train/train-faces/F0911/MID1/P09623_face0.jpg
1,Train/train-faces/F0782/MID1/P08258_face4.jpg
2,Train/train-faces/F0232/MID2/P02462_face0.jpg
3,Train/train-faces/F0089/MID1/P00918_face0.jpg
4,Train/train-faces/F0639/MID5/P06711_face0.jpg
...,...
12161,Train/train-faces/F0425/MID8/P11471_face6.jpg
12162,Train/train-faces/F0601/MID20/P11953_face2.jpg
12163,Train/train-faces/F0601/MID2/P06274_face2.jpg
12164,Train/train-faces/F0601/MID9/P06244_face1.jpg


In [2]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
Image.MAX_IMAGE_PIXELS = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='001', help='path to the config file.')
parser.add_argument('--vgg_model_path', type=str, default='./models/dex_imdb_wiki.caffemodel.pt', help='pretrained age classifier')
parser.add_argument('--log_path', type=str, default='./logs/', help='log file path')
parser.add_argument('--multigpu', type=bool, default=False, help='use multiple gpus')
parser.add_argument('--checkpoint', type=str, default='', help='checkpoint file path')
parser.add_argument('--img_path', type=str, default='./test/input/', help='test image path')
parser.add_argument('--out_path', type=str, default='./test/output/', help='test output path')
parser.add_argument('--target_age', type=int, default=55, help='Age transform target, interger value between 20 and 70')
opts = parser.parse_known_args()[0]

log_dir = os.path.join(opts.log_path, opts.config) + '/'
if not os.path.exists(opts.out_path):
    os.makedirs(opts.out_path)

config = yaml.safe_load(open('./configs/' + opts.config + '.yaml', 'r'))
img_size = (config['input_w'], config['input_h'])

# Initialize trainer
trainer = Trainer(config)
device = torch.device('cuda')
trainer.to(device)

# Load pretrained model 
if opts.checkpoint:
    trainer.load_checkpoint(opts.checkpoint)
else:
    trainer.load_checkpoint(log_dir + 'checkpoint')

def preprocess(img_name):
    resize = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor()
            ])
    normalize = transforms.Normalize(mean=[0.48501961, 0.45795686, 0.40760392], std=[1,1,1])
    img_pil = Image.open(opts.img_path + img_name)
    img_np = np.array(img_pil)
    img = resize(img_pil)
    if img.size(0) == 1:
        img = torch.cat((img, img, img), dim = 0)
    img = normalize(img)
    return img

In [4]:
target_age = 60

target_image = '48517.png'

with torch.no_grad():
    img = preprocess(target_image).unsqueeze(0).to(device)

    age_modif = torch.tensor(target_age).unsqueeze(0).to(device)
    img_modif = trainer.test_eval(img, age_modif, target_age=target_age, hist_trans=True)  
    utils.save_image(clip_img(img_modif), opts.out_path + target_image.split('.')[0] + '_age_' + str(target_age) + '.jpg')

In [40]:
# Set target age
target_age = 60

# Load test image
img_list = os.listdir(opts.img_path)
img_list.sort()

with torch.no_grad():
    for img_name in img_list:
        image_A = preprocess(img_name)
        image_A = image_A.unsqueeze(0).to(device)

        age_modif = torch.tensor(target_age).unsqueeze(0).to(device)
        image_A_modif = trainer.test_eval(image_A, age_modif, target_age=target_age, hist_trans=True)  
        utils.save_image(clip_img(image_A_modif), opts.out_path + img_name.split('.')[0] + '_age_' + str(target_age) + '.jpg')

        # Plot manipulated image
        img_out = np.array(Image.open(opts.out_path + img_name.split('.')[0] + '_age_' + str(target_age) + '.jpg'))
        plt.axis('off')
        plt.imshow(img_out)
        plt.show() 

TypeError: pic should be PIL Image or ndarray. Got <class 'str'>

In [34]:
from nets import *
from functions import *
import pandas as pd

CACD_PATH = '/mnt/heavy/DeepLearning/Datasets/CACD2000/'

age_classifier = VGG()
cacd = pd.read_csv(CACD_PATH + 'celebrity2000_meta.csv')
cacd

,age,id,year_taken,rank,lfw,birth,name
0,53,1,2004,1,1,1951,53_Robin_Williams_0001.jpg
1,53,1,2004,1,1,1951,53_Robin_Williams_0002.jpg
2,53,1,2004,1,1,1951,53_Robin_Williams_0003.jpg
3,53,1,2004,1,1,1951,53_Robin_Williams_0004.jpg
4,53,1,2004,1,1,1951,53_Robin_Williams_0005.jpg
...,...,...,...,...,...,...,...
163441,23,2000,2013,50,0,1990,23_Katie_Findlay_0009.jpg
163442,23,2000,2013,50,0,1990,23_Katie_Findlay_0010.jpg
163443,23,2000,2013,50,0,1990,23_Katie_Findlay_0011.jpg
163444,23,2000,2013,50,0,1990,23_Katie_Findlay_0012.jpg


In [35]:
def get_age(path):
    img = Image.open(CACD_PATH + '53_Robin_Williams_0001.jpg')

    # Preprocess the image
    preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.48501961, 0.45795686, 0.40760392], std=[1, 1, 1]),
            ])

    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension

    predict_age_pb = age_classifier(vgg_transform(img_tensor))['fc8']

    # Get predicted age
    predict_age = get_predict_age(predict_age_pb)
    print(predict_age)
    return int(predict_age)

In [38]:
import tqdm

ages = []
# iterrows with tqdm
for i, row in tqdm.tqdm(cacd.head(5).iterrows()):
    ages.append(get_age(row['name']))

2it [00:00,  7.65it/s]

tensor([49.9818], grad_fn=<CopySlices>)
tensor([49.9818], grad_fn=<CopySlices>)


4it [00:00,  7.37it/s]

tensor([49.9818], grad_fn=<CopySlices>)
tensor([49.9818], grad_fn=<CopySlices>)


5it [00:00,  7.42it/s]

tensor([49.9818], grad_fn=<CopySlices>)


In [27]:
ages

[49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49]